# Security-Constrained Optimisation

In this example, the dispatch of generators is optimised using the security-constrained linear OPF, to guaranteed that no branches are overloaded by certain branch outages.

In [ ]:
import numpy as np

import pypsa

In [ ]:
network = pypsa.examples.scigrid_de(from_master=True)

There are some infeasibilities without line extensions.

In [ ]:
for line_name in ["316", "527", "602"]:
    network.lines.loc[line_name, "s_nom"] = 1200

now = network.snapshots[0]

Performing security-constrained linear OPF

In [ ]:
branch_outages = network.lines.index[:15]
network.optimize.optimize_security_constrained(now, branch_outages=branch_outages)

For the PF, set the P to the optimised P.

In [ ]:
network.generators_t.p_set = network.generators_t.p_set.reindex(
    columns=network.generators.index
)
network.generators_t.p_set.loc[now] = network.generators_t.p.loc[now]

network.storage_units_t.p_set = network.storage_units_t.p_set.reindex(
    columns=network.storage_units.index
)
network.storage_units_t.p_set.loc[now] = network.storage_units_t.p.loc[now]

Check no lines are overloaded with the linear contingency analysis

In [ ]:
p0_test = network.lpf_contingency(now, branch_outages=branch_outages)
p0_test

Check loading as per unit of s_nom in each contingency

In [ ]:
max_loading = (
    abs(p0_test.divide(network.passive_branches().s_nom, axis=0)).describe().loc["max"]
)
max_loading

In [ ]:
np.allclose(max_loading, np.ones(len(max_loading)))